In [1]:
import torch
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import torch.nn as nn
import numpy as np

In [2]:
from hpelm import ELM

**Read data**

In [3]:
df=pd.read_csv("/Users/rahuljauhari/Desktop/research runoff/final destination/merged_imd.csv")
df.drop(columns=['Unnamed: 0'],inplace=True)

In [4]:
df['DateTime'] = pd.to_datetime(df['DateTime'])
df.set_index('DateTime', inplace=True)

In [5]:
monthly_mean = df.resample('M').mean()

**Actual value**

In [6]:
df_actual=pd.read_excel("/Users/rahuljauhari/Desktop/research runoff/Calibrated and Validated.xlsx")

In [7]:
# select last column
observed_runnoff=df_actual['observed']

**Normalization**

In [8]:
from scipy.stats import zscore
def func(name):
    x=0
    y=0
    inv= 0
    if name=='zscore':
        x_norm = zscore(monthly_mean)
        y_norm = zscore(observed_runnoff)
        x_norm[x_norm > 3] = 2.8
        x_norm[x_norm < -3] = -2.8
        y_norm[y_norm >3] = 2.8
        y_norm[y_norm < -3] = -2.8
        x=x_norm
        y=y_norm
    if name=='StandardScaler':
        scaler = StandardScaler()
        x_scaled = scaler.fit_transform(monthly_mean)
        y_scaled = scaler.fit_transform(observed_runnoff.values.reshape(-1,1))
        x_scaled[x_scaled > 3] = 2.8
        x_scaled[x_scaled < -3] = -2.8
        y_scaled[y_scaled >3] = 2.8
        y_scaled[y_scaled < -3] = -2.8
        x=      x_scaled  
        y=y_scaled
        inv = scaler
        
    if name == 'MinMaxScaler':
        scaler = MinMaxScaler(feature_range=(0,1))
        x_scaled = scaler.fit_transform(monthly_mean)
        y_scaled = scaler.fit_transform(observed_runnoff.values.reshape(-1,1))
        x=      x_scaled  
        y=y_scaled
        inv = scaler
    return x,y,inv

In [9]:

from sklearn.metrics import mean_squared_error
def rmse1(yt, yp): #lower the better
    return np.sqrt(mean_squared_error(yt, yp))
# Kling-Gupta effciency
def kge1(yt, yp): #highqer the better
    r = np.corrcoef(yt, yp,rowvar=False)[0, 1]
    alpha = np.std(yp) / np.std(yt)
    beta = np.mean(yp) / np.mean(yt)
    return 1 - np.sqrt((r - 1)**2 + (alpha - 1)**2 + (beta - 1)**2)
    # r squared
def r21(yt, yp): #higher the better
    return 1 - np.sum((yt - yp)**2) / np.sum((yt - np.mean(yt))**2)
    # Nash-Sutcliffe efficiency
def nse(predictions, targets):
    return (1-(np.sum((predictions-targets)**2)/np.sum((targets-np.mean(targets))**2)))


In [10]:
from sklearn.model_selection import train_test_split
x,y,inv_scaler= func('MinMaxScaler')
X_train, X_test,y_train,y_test = train_test_split(x,y,test_size=0.3,shuffle=False)

In [11]:
# “lin” for linear, “sigm” or “tanh” for non-linear, “rbf_l1”, “rbf_l2” or “rbf_linf” for radial basis function neurons.
act = 'tanh'
neurons = 64
activation = 'relu'
elm = ELM(X_train.shape[1], y_train.shape[1],activation,batch=5)
elm.add_neurons(neurons, act,)
elm.add_neurons(neurons, act)
elm.train(X_train, y_train, 'faster2')
Y_pred_train = elm.predict(X_train)
Y_pred_test = elm.predict(X_test)

In [12]:
print("Training RMSE: ", rmse1(y_train, Y_pred_train))
print("Testing RMSE: ", rmse1(y_test, Y_pred_test))
print("Training KGE: ", kge1(y_train, Y_pred_train))
print("Testing KGE: ", kge1(y_test, Y_pred_test))
print("Training R2: ", r21(y_train, Y_pred_train))
print("Testing R2: ", r21(y_test, Y_pred_test))
print("Training NSE: ", nse(y_train, Y_pred_train))
print("Testing NSE: ", nse(y_test, Y_pred_test))

Training RMSE:  0.05584857938017681
Testing RMSE:  0.11110732239092555
Training KGE:  0.9117697001985273
Testing KGE:  0.6178533433523663
Training R2:  0.8791177736093172
Testing R2:  0.4995441495977966
Training NSE:  0.8624953663974426
Testing NSE:  0.2275679520693934
